In [1]:
import pandas as pd
import requests
from io import StringIO

# ----------------------------
# CO-OPS settings
# ----------------------------
BASE = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
STATION = "8654467"

START = pd.Timestamp("2010-04-01")
END   = pd.Timestamp.today().normalize()

OUTCSV = f"COOPS_{STATION}_wind_metric_GMT.csv"

# ----------------------------
# Fetch one chunk
# ----------------------------
def fetch_chunk(beg, fin, product="wind", units="metric", time_zone="gmt"):
    params = dict(
        product=product,
        application="bulk_download_script",
        begin_date=beg.strftime("%Y%m%d"),
        end_date=fin.strftime("%Y%m%d"),
        station=STATION,
        units=units,
        time_zone=time_zone,
        format="csv",
    )

    r = requests.get(BASE, params=params, timeout=60)
    r.raise_for_status()

    txt = r.text.strip()
    if not txt or txt.lower().startswith("error"):
        return None

    df = pd.read_csv(StringIO(txt))
    if df.empty:
        return None

    return df

# ----------------------------
# Download and concatenate
# ----------------------------
chunks = []

for beg in pd.date_range(START, END, freq="MS"):
    fin = beg + pd.offsets.MonthEnd(0)
    if fin > END:
        fin = END

    df = fetch_chunk(beg, fin)
    if df is not None:
        chunks.append(df)

if not chunks:
    raise RuntimeError("No wind data returned for this station.")

wind = pd.concat(chunks, ignore_index=True)

# ----------------------------
# Clean up columns
# ----------------------------
wind.columns = [c.strip() for c in wind.columns]

# Parse time
wind["Date Time"] = pd.to_datetime(wind["Date Time"], errors="coerce")

# Normalize numeric columns
for c in wind.columns:
    if c.lower().startswith(("wind", "gust")):
        wind[c] = pd.to_numeric(wind[c], errors="coerce")

wind = wind.dropna(subset=["Date Time"]).sort_values("Date Time").reset_index(drop=True)

# ----------------------------
# Save
# ----------------------------
wind.to_csv(OUTCSV, index=False)

print(f"Wrote {len(wind):,} rows to: {OUTCSV}")
print(wind.head())
print(wind.tail())


Wrote 1,378,872 rows to: COOPS_8654467_wind_metric_GMT.csv
            Date Time  Speed  Direction Direction.1  Gust  X  R
0 2010-04-01 00:00:00    NaN        NaN         NaN   NaN  1  1
1 2010-04-01 00:06:00    NaN        NaN         NaN   NaN  1  1
2 2010-04-01 00:12:00    NaN        NaN         NaN   NaN  1  1
3 2010-04-01 00:18:00    NaN        NaN         NaN   NaN  1  1
4 2010-04-01 00:24:00    NaN        NaN         NaN   NaN  1  1
                  Date Time  Speed  Direction Direction.1  Gust  X  R
1378867 2025-12-23 15:48:00    2.3       19.0         NNE   2.7  0  0
1378868 2025-12-23 15:54:00    2.8        5.0           N   3.0  0  0
1378869 2025-12-23 16:00:00    2.7       12.0         NNE   3.2  0  0
1378870 2025-12-23 16:06:00    2.3       26.0         NNE   3.2  0  0
1378871 2025-12-23 16:12:00    2.1       23.0         NNE   2.4  0  0
